# YouTube Trending Videos Analysis PJT

## In this project, I will be tracking YouTube trending videos from the trend tab, and create a Tableau dashboard.

# 1. Introduction
## 1.1 Background
YouTube has a trending tab that shows the trending videos in certain country. Since YouTube is a leading content platform for many countries, we can spot content trends just by observing trending videos in different countries. I wanted to create a Tableau dashboard that tracks trending videos for three countries (Korea, Japan, and USA), and use it to analyze content trends in 3 different countries. The scope of the project will be limited to the analysis of these 3 countries' weekly-updated trending videos. 

## 1.2 Objectives
In this project, I will focus on learning the followings:

- Get familiar with YouTube API, and use the API to gather YouTube trending videos
- Use cron to update data weekly
- Use Tableau to create a interactive dashboard
- Analyze trending videos for each country to find out what type of contents are popular in different countries:
    - What type of content is the hot trend right now in Korea, Japan, and USA?
    - What content format is gaining more popularity?
    - What content format is not as popular?
    - What channels entered the trending chart the most?

## 1.3 Project process
1. Get the trending video data for each country using YouTube Data API v3, and update the data weekly via cron
2. Preprocess data and engineer new features
3. Create a Tableau dashboard
4. Analyze data with Tableau dashboard

## 1.4 Dataset
### Data Source
For this project, I obtained the dataset myself by utilizing YouTube Data API v3. 

### Data Limitation
The data is a real-world dataset, suitable for research purposes. However, considering the API quota limit of 10,000 units per day, I am getting trending videos for just 3 countries (Korea, Japan, USA). Also, the data will be updated every week, since the trending chart doesn't change much on a daily basis. 

### Ethics of data source
According to the Youtube API guide, the usage of YouTube API is free and open to anyone who created API KEY. As long as the API user abides by the YouTube API quota limit, there is no issue in using YouTube API to get data. Also, the data itself is public data that can be obtained from the YouTube channel, so there is no privacy issue involved with the data source.


In [4]:
# Import basic libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import os

# Import API related libraries
from googleapiclient.discovery import build
from IPython.display import JSON

# Import API KEY from the config file
import sys
sys.path.append('/Users/minguyeo/Documents/coding/pythonPJT/config')
import yt_api_key as api

# Enable Korean Font
from matplotlib import font_manager, rc
import platform

if platform.system() == 'Windows':
# For windows user
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
# For mac user
    rc('font', family='AppleGothic')

plt.rcParams['axes.unicode_minus'] = False

# Import google sheet
import pygsheets
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Access the environment variables
google_sheets_creds = os.getenv('GOOGLE_SHEETS_CREDS')

# Use the environment variable
if google_sheets_creds:
    gc = pygsheets.authorize(service_account_file=google_sheets_creds)
else:
    print("Environment variable GOOGLE_SHEETS_CREDS is not set.")

# Get google sheet data
spreadsheet_url1 = "https://docs.google.com/spreadsheets/d/1zxmjY48uOkyq-e6o-Fxc9FE_G98jZFM8wdHJ3lxqsoI/edit#gid=0"
sid1 = spreadsheet_url1.split("/d/")[1].split("/edit")[0]
sh1 = gc.open_by_key(sid1)

# Weekly YouTube Trend DB
wks_trend_data = sh1.worksheet_by_title('yt_trend_weekly')

# Sheet data to dataframe
trend_df = wks_trend_data.get_as_df(has_header=True , index_column=None , start='A1', end='AE1000000' , numerize=True , empty_value=np.nan)


# 2. Data collection with YouTube Data API v3

First, I created an API key from the google cloud platform(GCP) console, and enabled YouTube Data API v3 for my account. I saved the API_KEY in the separate config directory, so I can import the API_KEY without showing the key in the notebook. Then, I checked the YouTube Data API documentation to find out how to get trending videos for each region (Korea, Japan, USA). Then created a `get_trending_video` function to collect video statistics of trending videos for each country via the API. 

In [2]:
# Build API service
youtube = build('youtube', 'v3', developerKey=api.API_KEY)

regions = ['KR', 'JP', 'US']

The region code for three countries:
- Korea = KR
- Japan = JP
- United States = US

In [3]:
# Find the region code for Korea, Japan, USA
request = youtube.i18nRegions().list(
        part="snippet",
        hl="en_US"
    )
response = request.execute()
print(response['items'])

[{'kind': 'youtube#i18nRegion', 'etag': 'RmvQrICcSKx5BZkuwuIam0hkr9o', 'id': 'AE', 'snippet': {'gl': 'AE', 'name': 'United Arab Emirates'}}, {'kind': 'youtube#i18nRegion', 'etag': 'lYa87JjqaXmfqWaf6vuQD3YSNo4', 'id': 'BH', 'snippet': {'gl': 'BH', 'name': 'Bahrain'}}, {'kind': 'youtube#i18nRegion', 'etag': 'sNy_f3n1a8hzh4cy6lk7Blq6MgU', 'id': 'DZ', 'snippet': {'gl': 'DZ', 'name': 'Algeria'}}, {'kind': 'youtube#i18nRegion', 'etag': 'epyyUiiXSMjhLjkCvWGK-3zDKpM', 'id': 'EG', 'snippet': {'gl': 'EG', 'name': 'Egypt'}}, {'kind': 'youtube#i18nRegion', 'etag': 'foRr-koDaxI_23FCQgzB6S9YdQw', 'id': 'IQ', 'snippet': {'gl': 'IQ', 'name': 'Iraq'}}, {'kind': 'youtube#i18nRegion', 'etag': 'm1XLnpk2rbqMpwSZoka9-5P8gLc', 'id': 'JO', 'snippet': {'gl': 'JO', 'name': 'Jordan'}}, {'kind': 'youtube#i18nRegion', 'etag': 'zC0vjycew30h6Q1SVGY-wPYNOrg', 'id': 'KW', 'snippet': {'gl': 'KW', 'name': 'Kuwait'}}, {'kind': 'youtube#i18nRegion', 'etag': 'DJkocR6JpoY3ReMH479TGJMk-yk', 'id': 'LB', 'snippet': {'gl': 'LB'

In [41]:
def get_video_stats(youtube, country, page_token=None):
    stats = {
        'snippet': ['channelId', 'channelTitle', 'title', 'publishedAt', 'description', 'thumbnails', 'tags'],
        'contentDetails': ['duration'],
        'statistics': ['viewCount', 'likeCount', 'commentCount']
    }

    request = youtube.videos().list(
        part="snippet,contentDetails,statistics",
        chart="mostPopular",
        maxResults=50,
        regionCode=country,
        pageToken=page_token
    )

    response = request.execute()
    all_video_stat = []

    for video in response['items']:
        video_stat = {
            'region': country,
            'video_id': video['id']
        }

        for i in stats.keys():
            for k in stats[i]:
                try:
                    video_stat[k] = video[i][k]
                except:
                    video_stat[k] = None

        all_video_stat.append(video_stat)

    return all_video_stat, response.get('nextPageToken')

def get_trending_video(youtube, regions):
    all_video_stat = []

    for country in regions:
        video_stats, next_page_token = get_video_stats(youtube, country)

        while next_page_token:
            video_stats, next_page_token = get_video_stats(youtube, country, next_page_token)

        all_video_stat.extend(video_stats)

    return pd.DataFrame(all_video_stat)


In [42]:
trending_vid = get_trending_video(youtube, regions)

In [43]:
trending_vid.head()

,region,video_id,channelId,channelTitle,title,publishedAt,description,thumbnails,tags,duration,viewCount,likeCount,commentCount
0,KR,3NxJrDjSBw4,UCOszRhGW_mW8yPYKhLuULMw,푸먹_foomuk,무뚝뚝한 집안의 아기! / Baby from an expressionless fam...,2023-11-25T06:30:14Z,오늘은 로기가 태어나던 시절의 이야기입니다.\n그럼 오늘도 재밌게 봐주세요~ q(≧...,{'default': {'url': 'https://i.ytimg.com/vi/3N...,"[애니메이션먹방, ani mukbang, animation mukbang, asmr...",PT3M43S,1645852,24642,663
1,KR,j1BrwfiwLgI,UCK1sVuXaDvJeNwl9noN5nOA,KBS Entertain,박진영 (Park Jin young) sweet dreams 외 3개 - 축하공연...,2023-11-24T14:23:35Z,박진영 (Park Jin young) sweet dreams+when we disc...,{'default': {'url': 'https://i.ytimg.com/vi/j1...,"[청룡영화상, 청룡영화제, 청룡, 시상식, 영화제, 남우주연상, 여우주연상, 김혜수...",PT5M16S,3001829,37563,10939
2,KR,yMUsVYIwwYE,UCLqecwv-NjFSJ4gNwUunhTA,MBN Drama,"[엠P!CK] 드디어 모든 진실이 밝혀지고, 끝까지 발악하다 집에서 쫓겨나는 이정혜...",2023-11-27T05:36:06Z,MBN 주말 미니시리즈 ＜완벽한 결혼의 정석＞ l 회귀 로맨스 복수극\n\n☞ MB...,{'default': {'url': 'https://i.ytimg.com/vi/yM...,"[주말 미니시리즈, 성훈, 진지희, 정유민, 강신효, 이민영, 전노민, 이미숙, 김...",PT11M47S,692607,4502,81
3,KR,fJRjCdd1nSs,UC9cCBxBAQW2CzLYeT20q49A,지식해적단,💀 이스라엘은 왜 헤즈볼라를 무서워할까? / 💀 헤즈볼라는 누구인가,2023-11-25T03:15:09Z,🏴‍☠️지식해적단 멤버십 가입\nhttps://www.youtube.com/chan...,{'default': {'url': 'https://i.ytimg.com/vi/fJ...,"[지식채널, 지식채널e, 역사채널, 중동문제, 팔레스타인, 아라파트, 팔랑헤, 마론...",PT19M55S,347886,5368,620
4,KR,X8QLyVf7cgM,UCX3jMYyi3smD-ltueIQMawA,산골여행,겨울에 더 맛있고 잘자라는 12월에 심어도 대박나는 8가지 작물 김장채소 뽑은자리에...,2023-11-24T08:55:59Z,,{'default': {'url': 'https://i.ytimg.com/vi/X8...,[km_20231123_1080p_30f_20231124_171322],PT23M56S,563153,8129,145


In [44]:
trending_vid.columns

Index(['region', 'video_id', 'channelId', 'channelTitle', 'title',
       'publishedAt', 'description', 'thumbnails', 'tags', 'duration',
       'viewCount', 'likeCount', 'commentCount'],
      dtype='object')

### Change column names

In [45]:
columns = {'channelId':'channel_id','channelTitle':'channel_name','publishedAt':'upload_date','viewCount':'view','likeCount':'like','commentCount':'comment'}
trending_vid = trending_vid.rename(columns = columns)

### Check for null, empty values

Tags, like, comment are the columns with null values. Nothing has to be done for null, empty values.

In [46]:
trending_vid.isnull().sum()

region           0
video_id         0
channel_id       0
channel_name     0
title            0
upload_date      0
description      0
thumbnails       0
tags            24
duration         0
view             0
like             1
comment          2
dtype: int64

### Reformat values and change data types

Change duration, upload_date format to datetime format

In [47]:
# Change upload_date column to datetime object
trending_vid['upload_date'] = pd.to_datetime(trending_vid['upload_date']).dt.tz_convert(None)

# Convert duration (isodate format) to datetime format
import isodate
trending_vid['duration'] =  trending_vid['duration'].apply(lambda x: isodate.parse_duration(x))

# Change columns with number values to int type
trending_vid[['view','like','comment']] = trending_vid[['view','like','comment']].apply(pd.to_numeric)

# Change description, and title to string type
trending_vid['description'] = trending_vid['description'].astype(str)
trending_vid['title'] = trending_vid['title'].astype(str)

In [48]:
trending_vid.head()

,region,video_id,channel_id,channel_name,title,upload_date,description,thumbnails,tags,duration,view,like,comment
0,KR,3NxJrDjSBw4,UCOszRhGW_mW8yPYKhLuULMw,푸먹_foomuk,무뚝뚝한 집안의 아기! / Baby from an expressionless fam...,2023-11-25 06:30:14,오늘은 로기가 태어나던 시절의 이야기입니다.\n그럼 오늘도 재밌게 봐주세요~ q(≧...,{'default': {'url': 'https://i.ytimg.com/vi/3N...,"[애니메이션먹방, ani mukbang, animation mukbang, asmr...",0 days 00:03:43,1645852,24642.0,663.0
1,KR,j1BrwfiwLgI,UCK1sVuXaDvJeNwl9noN5nOA,KBS Entertain,박진영 (Park Jin young) sweet dreams 외 3개 - 축하공연...,2023-11-24 14:23:35,박진영 (Park Jin young) sweet dreams+when we disc...,{'default': {'url': 'https://i.ytimg.com/vi/j1...,"[청룡영화상, 청룡영화제, 청룡, 시상식, 영화제, 남우주연상, 여우주연상, 김혜수...",0 days 00:05:16,3001829,37563.0,10939.0
2,KR,yMUsVYIwwYE,UCLqecwv-NjFSJ4gNwUunhTA,MBN Drama,"[엠P!CK] 드디어 모든 진실이 밝혀지고, 끝까지 발악하다 집에서 쫓겨나는 이정혜...",2023-11-27 05:36:06,MBN 주말 미니시리즈 ＜완벽한 결혼의 정석＞ l 회귀 로맨스 복수극\n\n☞ MB...,{'default': {'url': 'https://i.ytimg.com/vi/yM...,"[주말 미니시리즈, 성훈, 진지희, 정유민, 강신효, 이민영, 전노민, 이미숙, 김...",0 days 00:11:47,692607,4502.0,81.0
3,KR,fJRjCdd1nSs,UC9cCBxBAQW2CzLYeT20q49A,지식해적단,💀 이스라엘은 왜 헤즈볼라를 무서워할까? / 💀 헤즈볼라는 누구인가,2023-11-25 03:15:09,🏴‍☠️지식해적단 멤버십 가입\nhttps://www.youtube.com/chan...,{'default': {'url': 'https://i.ytimg.com/vi/fJ...,"[지식채널, 지식채널e, 역사채널, 중동문제, 팔레스타인, 아라파트, 팔랑헤, 마론...",0 days 00:19:55,347886,5368.0,620.0
4,KR,X8QLyVf7cgM,UCX3jMYyi3smD-ltueIQMawA,산골여행,겨울에 더 맛있고 잘자라는 12월에 심어도 대박나는 8가지 작물 김장채소 뽑은자리에...,2023-11-24 08:55:59,,{'default': {'url': 'https://i.ytimg.com/vi/X8...,[km_20231123_1080p_30f_20231124_171322],0 days 00:23:56,563153,8129.0,145.0


### Engineer new features

Get thumbnail image url and week number

In [49]:
# Get medium quality image url from each thumbnails column 
trending_vid['thumbnail_url'] = trending_vid['thumbnails'].apply(lambda x: x['medium']['url'])

# Drop original thumbnails column
trending_vid = trending_vid.drop('thumbnails', axis=1)

In [50]:
# Get current year, week info and save it in column year, and week
trending_vid['year'] = datetime.datetime.today().year
trending_vid['week'] = 'W'+str(datetime.datetime.today().isocalendar()[1])

Create a `rank` column that contains the video's rank from the trending chart for each region

In [51]:
# Get rank value by using the index number
trending_vid = trending_vid.reset_index()
trending_vid['rank'] = trending_vid.groupby('region')['index'].rank(method='dense').astype(int)
trending_vid = trending_vid.drop('index', axis=1)

### Save cleaned data to spreadsheet (weekly)

In [52]:
# Combine original dataframe with updated data
final_df = pd.concat([trend_df, trending_vid])

# Save the combined dataframe to google sheet
wks_trend_data = sh1.worksheet_by_title('yt_trend_weekly')
wks_trend_data.clear('A1','AZ')
wks_trend_data.set_dataframe(final_df, 'A1', index=False)